<a href="https://colab.research.google.com/github/lucken99/ConstitutionXpert/blob/main/ProjectCI_Aug26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Install all required libraries
!pip install langchain
!pip install pypdf

!pip install pdfminer.six
# !pip install unstructured pdf2image  # for unstructured pdf loader (legacy)

!pip install tiktoken
!pip install sentence_transformers
!pip install chromadb
!pip install cohere
!pip install openai

# DATA

> We have text file which consists of paragraphs related to Indian Constitution for e.g., Articles, Schedules, etc.

>

In [2]:
# data path
dir_path = "/content/drive/MyDrive/Project_CILLM/db/"
text_file_path = "/content/drive/MyDrive/Project_CILLM/db/file_context_corpus_cleaned_extended_part3.txt"
text_file_path = "/content/drive/MyDrive/Project_CILLM/colab_project/Constitution-Xpert using OpenAI Embeddings (1)/file_context_corpus_cleaned_extended_part3.txt"
pdf_file_path = "/content/drive/MyDrive/Project_CILLM/db/file_context_corpus_cleaned_extended_part3.pdf"


In [3]:
# utility function to load api keys from json file
import json
path_to_keys = "/content/drive/MyDrive/Project_CILLM/Keys/keys.json"
def return_api_key(name):
    with open(path_to_keys, 'r') as f:
        json_data = json.load(f)
        return json_data[name]

# RETRIEVAL


## [Document loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/)

In [4]:
# Text Loaders
from langchain.document_loaders import TextLoader

# PDF Loaders (try which suits us best)
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
# from langchain.document_loaders import MathpixPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from langchain.document_loaders import AmazonTextractPDFLoader
from langchain.document_loaders import OnlinePDFLoader

loader_text = TextLoader(text_file_path)
loader_pdf = PyPDFLoader(pdf_file_path)

In [5]:
## utility function for document loaders information

def loaded_doc_info(loader, show_loaded_data=False):
    data = loader.load()
    print("Type of the loader:", type(loader))
    print("Length of the data:", len(data))
    if show_loaded_data:
        print(data)
    return data




In [6]:
# loaded_doc_info(loader_text, show_loaded_data=True)
text_data = loaded_doc_info(loader_text)

Type of the loader: <class 'langchain.document_loaders.text.TextLoader'>
Length of the data: 1


In [7]:
pdf_data = loaded_doc_info(loader_pdf)

Type of the loader: <class 'langchain.document_loaders.pdf.PyPDFLoader'>
Length of the data: 178


In [ ]:
# # checking different pdf loader
# loader = PDFMinerPDFasHTMLLoader(pdf_file_path)
# data = loaded_doc_info(loader, True)

## [Document Transformers](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

In [8]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter  # we can split by parts in constitution using this splitter and add metadata for better search
                                                                # first we have to add headers (for e.g., # or ##)

# # Split by tokens
# from langchain.text_splitter import TokenTextSplitter
# from langchain.text_splitter import SpacyTextSplitter
# from langchain.text_splitter import NLTKTextSplitter

# !pip install tiktoken
# good for OpenAI Models
# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=100, chunk_overlap=0
# )

# # Sentence Transformers token split
# from langchain.text_splitter import SentenceTransformersTokenTextSplitter # for a particular sentence transformer

# # Hugging face tokenizers
# from transformers import GPT2TokenizerFast
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
#     tokenizer, chunk_size=100, chunk_overlap=0
# )
# texts = text_splitter.split_text(text)


In [9]:
def split_data(loader, splitter):
    docs = loader.load_and_split(splitter)
    print("Loader Type:", type(loader))
    print("Splitter Type:", type(splitter))
    print("Length of splitted data:", len(docs))
    return docs




In [10]:
# using openai's tiktoken for splitting data
text_splitter = CharacterTextSplitter.from_tiktoken_encoder()
docs_text = split_data(loader_text, text_splitter)
docs_pdf = split_data(loader_pdf, text_splitter)

Loader Type: <class 'langchain.document_loaders.text.TextLoader'>
Splitter Type: <class 'langchain.text_splitter.CharacterTextSplitter'>
Length of splitted data: 31
Loader Type: <class 'langchain.document_loaders.pdf.PyPDFLoader'>
Splitter Type: <class 'langchain.text_splitter.CharacterTextSplitter'>
Length of splitted data: 178


In [11]:
# using RecursiveCharacterTextSplitter
recur_splitter = RecursiveCharacterTextSplitter()
docs_text = split_data(loader_text, recur_splitter)
docs_pdf = split_data(loader_pdf, recur_splitter)

Loader Type: <class 'langchain.document_loaders.text.TextLoader'>
Splitter Type: <class 'langchain.text_splitter.RecursiveCharacterTextSplitter'>
Length of splitted data: 156
Loader Type: <class 'langchain.document_loaders.pdf.PyPDFLoader'>
Splitter Type: <class 'langchain.text_splitter.RecursiveCharacterTextSplitter'>
Length of splitted data: 178


In [ ]:
docs_pdf[0]

In [12]:
char_splitter = CharacterTextSplitter(
    separator = "\n\n",
    # chunk_size = 1000,
    # chunk_overlap  = 200,
    # length_function = len,
    is_separator_regex = False,
)

# char_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=500)

docs_text = split_data(loader_text, char_splitter)
docs_pdf = split_data(loader_pdf, char_splitter)


Loader Type: <class 'langchain.document_loaders.text.TextLoader'>
Splitter Type: <class 'langchain.text_splitter.CharacterTextSplitter'>
Length of splitted data: 156
Loader Type: <class 'langchain.document_loaders.pdf.PyPDFLoader'>
Splitter Type: <class 'langchain.text_splitter.CharacterTextSplitter'>
Length of splitted data: 178


In [13]:
docs = [doc.page_content for doc in docs_text]

In [ ]:
# docs_pdf[0].page_content

In [ ]:
# print(docs_pdf[0].page_content.rstrip('\n'))

## Text Embedding models and chromadb Vectorstore
[MTEB blog](https://huggingface.co/blog/mteb) <br>
[MTEB](https://huggingface.co/spaces/mteb/leaderboard)

### Chromadb VectorStore


In [14]:
from langchain.vectorstores import Chroma


In [15]:
# utility function for embeddings info

def embeddings_info(embeddings):
    print("Total Embeddings:", len(embeddings))
    print("Dimension:", len(embeddings[0]))


In [16]:
### Problems with Chroma
### https://python.langchain.com/docs/integrations/vectorstores/chroma#basic-example-including-saving-to-disk
## Caching Embeddings using LocalFileStore
# from langchain.storage import LocalFileStore
# from langchain.embeddings import CacheBackedEmbeddings # Embeddings can be stored or temporarily cached to avoid needing to recompute them.
# import os

# utility function for creating embeddings using chromdb
# def create_cached_embeddings(docs, embedding_model, model_name, fs):
#     # print(list(fs.yield_keys()))
#     # cached_embedder = CacheBackedEmbeddings.from_bytes_store(
#     #     embedding_model, fs, namespace=model_name
#     # )
#     ### Create the vectorstore
#     # db = Chroma.from_documents(docs, cached_embedder)
#     db = Chroma.from_documents(docs, embedding_model)
#     return db

# def create_embeddings(docs, embedding_model):
#     return Chroma.from_documents(docs, embedding_model)


# utility function for similarity search
def return_similar_docs(db, query, k=4, show_docs=False):
    docs = db.similarity_search_with_relevance_scores(query, k=k)
    docs = sorted(docs, key=lambda x: -x[1])
    if show_docs:
        for doc in docs:
            print("Text:", doc[0].page_content)
            print("Relevance Score:", doc[1])
            print("--"*50)
            print("--"*50)
    return docs



### BGE Hugging face embeddings

In [19]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
# embeddings = hf.embed_documents(docs)
# embeddings_info(embeddings)
# sentence1 = "Tell me about article 23"
# embedding1 = hf.embed_query(sentence1)
# import numpy as np

# dot_product = np.dot(embeddings, embedding1)
# dot_product
# np.argmax()

In [20]:
hf_db = Chroma.from_documents(docs_text, hf, persist_directory="./chroma_db_hf")

In [21]:
### check similar documents
query = "What is writs in constitution?"
retrived_docs = return_similar_docs(hf_db, query, show_docs=True)

Text: CONSTITUTION OF INDIA:

The Constitution of India is the supreme law of India. The document lays down the framework that demarcates fundamental political code, structure, procedures, powers, and duties of government institutions and sets out fundamental rights, directive principles, and the duties of citizens. It is the longest written national constitution in the world.

The Constitution of India imparts constitutional supremacy (not parliamentary supremacy, since it was created by a constituent assembly rather than Parliament) and was adopted by its people with a declaration in its preamble.Parliament cannot override the constitution.

The Constitution was adopted by the Constituent Assembly of India on 26 November 1949 and became effective on 26 January 1950. The constitution replaced the Government of India Act 1935 as the country's fundamental governing document, and the Dominion of India became the Republic of India. To ensure constitutional autochthony, its framers repeale

### Cohere Embeddings

In [22]:
COHERE_API_KEY = return_api_key("COHERE_API_KEY")

In [23]:
from langchain.embeddings import CohereEmbeddings
cohr = CohereEmbeddings(
    model="embed-english-v2.0",
    cohere_api_key=COHERE_API_KEY
    )


In [24]:
# db_cohr = create_embeddings(docs_text, cohr)
cohr_db = Chroma.from_documents(docs_text, cohr, persist_directory="./chroma_db_cohr")

In [25]:
### check similar documents
query = "What is writs in constitution?"
retrived_docs = return_similar_docs(cohr_db, query, show_docs=True)

Text: Article 241: High Courts' power to issue certain writs
(1) The High Court shall have power, throughout the territories in relation to which it exercises jurisdiction, to issue to any person or authority, including in appropriate cases, any Government, directions, orders or writs, including writs in the nature of habeas corpus, mandamus, prohibition, quo warranto and certiorari, or any of them, for the enforcement of any of the rights conferred by Part III and for any other purpose.
(2) The power conferred by clause (1) shall not be in derogation of the power conferred on the Supreme Court by Article 32.

Article 242: Administration of tribal areas
(1) The President may, with the consent of the Parliament, by order, direct that any area in the States of Assam, Meghalaya, Tripura and Mizoram shall be administered as a tribal area.
(2) An order made under clause (1) may, from time to time, be modified or revoked by the President by a like order.
(3) The administration of a tribal ar

/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/base.py:254: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="Article 241: High Courts' power to issue certain writs\n(1) The High Court shall have power, throughout the territories in relation to which it exercises jurisdiction, to issue to any person or authority, including in appropriate cases, any Government, directions, orders or writs, including writs in the nature of habeas corpus, mandamus, prohibition, quo warranto and certiorari, or any of them, for the enforcement of any of the rights conferred by Part III and for any other purpose.\n(2) The power conferred by clause (1) shall not be in derogation of the power conferred on the Supreme Court by Article 32.\n\nArticle 242: Administration of tribal areas\n(1) The President may, with the consent of the Parliament, by order, direct that any area in the States of Assam, Meghalaya, Tripura and Mizoram shall be administered as a tribal 

### OpenAI Embeddings

In [ ]:
OPENAI_API_KEY = return_api_key("OPENAI_API_KEY")
# OPENAI_API_KEY

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

openai = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


#### We can also use [VectorstoreIndexCreator](https://python.langchain.com/docs/modules/data_connection/retrievers/#one-line-index-creation)
for creating vectorstore quickly with one liner code.

In [30]:
from langchain.indexes import VectorstoreIndexCreator

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=CohereEmbeddings(cohere_api_key=COHERE_API_KEY),
    text_splitter=CharacterTextSplitter(separator='\n\n'),
)

index = index_creator.from_loaders([loader_text])

# querying
query = "what is article 4?"
# index.query(query) # give your llm
# index.query_with_sources(query) # give your llm

# check Vectorstore
index.vectorstore

# can use as a retriever
index.vectorstore.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7a003b083ac0>, search_type='similarity', search_kwargs={})

## [Retrievers](https://python.langchain.com/docs/modules/data_connection/retrievers/)

### MultiQueryRetriever